# import

In [1]:
import os
import sys

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

sys.path.append('../src')
import utils

sys.path.append('./src')
from model import *

In [2]:
config = utils.readConfig('../config.json')
output_path = config['filepath']['output_dir'] + '/01'

utils.makeDirs(output_path, ['graph'])

# GPU or CPU

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# processing

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(), #0〜255の整数値を0〜1の浮動小数点数型に変換する
    transforms.Normalize((0.5,), (0.5,), (0.5,)) #平均と標準偏差に0.5を指定することで、値の範囲を[-1, 1]にする
])

#  load dataset & dataloder

In [5]:
test_dataset = datasets.CIFAR10(root=f"{output_path}/cifar_data", train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


# load model 

In [6]:
model = ResNet34()
model.load_state_dict(torch.load(f"{output_path}/model.pth"))

<All keys matched successfully>

# test

In [7]:
dataiter = iter(test_dataloader)
images, labels = dataiter.next()

outputs = model(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

correct = 0
total = 0
'''
**** 変更 ****
no_grad->inference_mode
こちらの方が早い
'''
with torch.inference_mode():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Predicted:    cat   car  deer plane
Accuracy of the network on the 10000 test images: 62 %


# total accuracy 

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))